In [0]:
! pip install simpletransformers

     |████████████████████████████████| 102kB 4.0MB/s 
     |████████████████████████████████| 194kB 12.7MB/s 
     |████████████████████████████████| 450kB 17.2MB/s 
     |████████████████████████████████| 1.0MB 8.9MB/s 
     |████████████████████████████████| 860kB 37.1MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=860f2be23baea247f815de27f4beb4ab61f9955188a1c7e9234a810da37f961e
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=70fcedc7d93ca6b6544f6ee36478bcea83c16590f88454d270fe34223699a00a
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built seqeval sacremoses


In [0]:
import dill
import pickle
import pandas as pd
import numpy as np
import shutil
import os

import random
import torch

from distutils.dir_util import copy_tree

from simpletransformers.classification import ClassificationModel

from google.colab import files

In [0]:
mkdir data

In [0]:
# mount my drive and save results:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
# Ohne der Funktion gibts keine reproduzierbaren Ergebnisse


def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    print("all seeds set")
  



set_seed(42)

all seeds set


In [0]:
#load files:
file_train = "/content/drive/My Drive/Master_Thesis/The_10_Folds/seneu_train_4.txt"
file = open(file_train, "rb")
trainfolds= pickle.load(file)

In [0]:
file_test = "/content/drive/My Drive/Master_Thesis/The_10_Folds/seneu_test_4.txt"
file = open(file_test, "rb")
testfolds= pickle.load(file)


In [0]:
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score
import sklearn

set_seed(42)

counter = 0

resultatos = []

while counter != len(trainfolds):

    set_seed(42)

    #Parameter to set:

    trainEpochs = 4
    learningRate = 2e-5




    model = ClassificationModel('bert', 'bert-base-multilingual-cased',args={'num_train_epochs': trainEpochs, 'learning_rate': learningRate, 'fp16': False, 
                                                                             'max_seq_length': 160, 'train_batch_size': 24,'eval_batch_size': 24 , 
                                                                             'warmup_ratio': 0.0,'weight_decay': 0.00, 
                                                                             'overwrite_output_dir': True})

    print("start with fold_{}".format(counter))
    trainfolds["{}_fold".format(counter)].to_csv("/content/data/train.tsv", sep="\t", index = False, header=False)
    print("{}_fold Train als train.tsv exportiert". format(counter))
    testfolds["{}_fold".format(counter)].to_csv("/content/data/dev.tsv", sep="\t", index = False, header=False)
    print("{}_fold test als train.tsv exportiert". format(counter))

    train_df =  pd.read_csv("/content/data/train.tsv", delimiter='\t', header=None)
    eval_df = df = pd.read_csv("/content/data/dev.tsv", delimiter='\t', header=None)

    train_df = pd.DataFrame({
    'text': train_df[3].replace(r'\n', ' ', regex=True),
    'label':train_df[1]})

    eval_df = pd.DataFrame({
    'text': eval_df[3].replace(r'\n', ' ', regex=True),
    'label':eval_df[1]})


    set_seed(42)

    model.train_model(train_df)

    set_seed(42)

    result, model_outputs, wrong_predictions = model.eval_model(eval_df, f1 = sklearn.metrics.f1_score, prec = sklearn.metrics.precision_score, acc =  sklearn.metrics.accuracy_score, rec = sklearn.metrics.recall_score)
    print(result)

    set_seed(42)

    resultatos.append(result)

    shutil.rmtree("outputs")
    shutil.rmtree("cache_dir")
    #shutil.rmtree("runs")



    counter += 1


all seeds set
all seeds set
start with fold_0
0_fold Train als train.tsv exportiert
0_fold test als train.tsv exportiert
all seeds set
Converting to features started.


Running loss: 0.451809

Running loss: 0.209003

Running loss: 0.192452

Running loss: 0.045997
Training of bert model complete. Saved to outputs/.
all seeds set
Converting to features started.


...

Running loss: 0.409555

Running loss: 0.601838

Running loss: 0.224537

Running loss: 0.130780
Training of bert model complete. Saved to outputs/.
all seeds set
Converting to features started.



{'mcc': 0.3167765125994181, 'tp': 161, 'tn': 72, 'fp': 94, 'fn': 28, 'f1': 0.7252252252252251, 'prec': 0.6313725490196078, 'acc': 0.6563380281690141, 'rec': 0.8518518518518519, 'eval_loss': 1.3234756390253704}
all seeds set


In [0]:
summ = 0
f1scr = 0
for i in resultatos:
  print( i["f1"])
  summ += i["f1"]

f1scr = summ / len(resultatos)
print("The F1 Score is:", f1scr)

0.7272727272727273
0.7499999999999999
0.7400468384074941
0.6735751295336787
0.6416040100250625
0.6579634464751959
0.6840855106888362
0.7242990654205608
0.6544502617801048
0.7252252252252251
The F1 Score is: 0.6978522214828885


In [0]:
#resultatos

In [0]:

summf1 = 0
f1scr = 0

summprec = 0
precscr = 0

summrec = 0
recscr = 0

summacc = 0
accscr = 0


for i in resultatos:
  summf1 += i["f1"]
  summprec += i["prec"]
  summrec += i["rec"]
  summacc += i["acc"]



f1scr = summf1 / len(resultatos)
precscr = summprec / len(resultatos)
recscr = summrec / len(resultatos)
accscr = summacc / len(resultatos)
print("The F1 Score is:", f1scr)
print("The Precision Score is:", precscr)
print("The Recall Score is:", recscr)
print("The Accuracy Score is:", accscr)


The F1 Score is: 0.6978522214828885
The Precision Score is: 0.6255757325959495
The Recall Score is: 0.7907388479628965
The Accuracy Score is: 0.6492957746478873


In [0]:
res_to_write = {"trainfile":file_train, "testfile":file_test,"The F1 Score is:": f1scr, "The Precision Score is:": precscr,"The Recall Score is:": recscr,"The Accuracy Score is:": accscr,"Epochs":trainEpochs,"learningRate":learningRate, "results": resultatos}

In [0]:
res_to_write

{'Epochs': 4,
 'The Accuracy Score is:': 0.6492957746478873,
 'The F1 Score is:': 0.6978522214828885,
 'The Precision Score is:': 0.6255757325959495,
 'The Recall Score is:': 0.7907388479628965,
 'learningRate': 2e-05,
 'results': [{'acc': 0.6535211267605634,
   'eval_loss': 1.3660170356432597,
   'f1': 0.7272727272727273,
   'fn': 35,
   'fp': 88,
   'mcc': 0.2843632263943768,
   'prec': 0.6507936507936508,
   'rec': 0.8241206030150754,
   'tn': 68,
   'tp': 164},
  {'acc': 0.7014084507042253,
   'eval_loss': 1.0472768664360046,
   'f1': 0.7499999999999999,
   'fn': 35,
   'fp': 71,
   'mcc': 0.3946103045678566,
   'prec': 0.691304347826087,
   'rec': 0.8195876288659794,
   'tn': 90,
   'tp': 159},
  {'acc': 0.6873239436619718,
   'eval_loss': 1.1956271608670552,
   'f1': 0.7400468384074941,
   'fn': 31,
   'fp': 80,
   'mcc': 0.375811024061814,
   'prec': 0.6638655462184874,
   'rec': 0.8359788359788359,
   'tn': 86,
   'tp': 158},
  {'acc': 0.6450704225352113,
   'eval_loss': 0.9355

In [0]:
file = open("/content/drive/My Drive/Master_Thesis/results/seneu4_4_2e5.txt", "wb")
pickle.dump(res_to_write, file)
file.close()

In [0]:
with open("/content/drive/My Drive/Master_Thesis/results/seneu4_4_2e5_readable.txt", "w") as f:
  print(res_to_write, file=f)
